# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [1]:
import numpy as np
import pandas as pd
import ipywidgets
import folium
from folium import plugins
from folium.plugins import MarkerCluster
import psycopg2
from sqlalchemy import create_engine
from getpass import getpass

### Abrindo dataset

In [3]:
data = pd.read_csv('../data/georef.csv').drop(columns=['Unnamed: 0'])

In [4]:
data.head()

,id_ocorrencia,conduta,natureza_lesao,tipo_crime,uf,municipio,endereco,logradouro_sede,geolocalizacao,latitude,longitude
0,2,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Impedir ou cercear o direito de reunião,RIO DE JANEIRO,Rio de Janeiro,Câmara Municipal,Câmara Municipal,0101000020E6100000E013EB54F91546C0DC847B65DE8A...,-22.542456,-44.171672
1,2,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Ameaçar a integridade física do indivíduo,RIO DE JANEIRO,Rio de Janeiro,Câmara Municipal,Câmara Municipal,0101000020E6100000E013EB54F91546C0DC847B65DE8A...,-22.542456,-44.171672
2,4,Operação policial - Repressão a manifestação,Crimes de lesão corporal (CP),Lesão corporal grave,DISTRITO FEDERAL,Brasília,Esplanada dos Ministérios,Esplanada dos Ministérios,0101000020E6100000EC53E98255EF47C0B71F95078E98...,-15.797959,-47.869797
3,4,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Impedir ou cercear o direito de reunião,DISTRITO FEDERAL,Brasília,Esplanada dos Ministérios,Esplanada dos Ministérios,0101000020E6100000EC53E98255EF47C0B71F95078E98...,-15.797959,-47.869797
4,4,Operação policial - Repressão a manifestação,Crimes de abuso de autoridade (Lei 4.898/1965),Ameaçar a integridade física do indivíduo,DISTRITO FEDERAL,Brasília,Esplanada dos Ministérios,Esplanada dos Ministérios,0101000020E6100000EC53E98255EF47C0B71F95078E98...,-15.797959,-47.869797


## Construindo o mapa 

1. Selecionar marcadores a partir de condição

IAN: por favor, implemente: 
 - Ao clicar no ícone abrir uma tabelinha com algumas informações do dataframe, não só o que eu determino na variável 'popup' (infos que quero: conduta / natureza_lesao / logradouro_sede)

### Definindo 'natureza_lesao' como objetivo de pesquisa

In [5]:
data['natureza_lesao'].unique();

In [6]:
# Pegando a média da latitudes e longitudes das ocorrências
latmean = data['latitude'].mean()
lonmean = data['longitude'].mean()

# Criando o mapa
mapa_cluster = folium.Map(location=[latmean,lonmean], 
                         zoom_start=8, 
                         tiles='OpenStreetMap', 
                         control_scale=True)

# Minimapa
minimap = plugins.MiniMap(toggle_display=True)
mapa_cluster.add_child(minimap)

# Plugins - fullscreen
plugins.Fullscreen(position='topleft').add_to(mapa_cluster);

### Definindo as cores

In [7]:
dic_nl = {'Crimes de abuso de autoridade (Lei 4.898/1965)':'pink',
          'Crimes de lesão corporal (CP)':'darkred',
          'Crimes contra a honra (CP)':'green',
          'Crimes contra a liberdade pessoal (CP)':'white',
          'Crimes contra a liberdade sexual (CP)':'purple',
          'Crimes contra a vida (CP)':'red',
          'Crimes contra a administração da justiça (CP)':'cadetblue',
          'Crimes de tortura (Lei 9.455/1997)':'black',
          'Crimes contra a administração em geral (CP)':'lightgreen',
          'Crimes contra a paz pública (CP)':'darkgreen',
          'Crimes de periclitação da vida e da saúde (CP)':'darkpurple',
          'Crimes de dano (CP)':'lightred',
          'Crimes de perigo comum (CP)':'orange',
          'Abuso de autoridade do ECA (Lei 8.069/1990)':'darkblue',
          'Crime de extorsão (CP)':'gray',
          'Crimes contra a fé pública (CP)':'blue',
          'Convenção contra o crime de genocídio':'beige'}

### Plotando sub-grupos em camadas

In [9]:
#adicionando clusterizacao
mc = MarkerCluster(control=False)
mapa_cluster.add_child(mc)

#definindo a funcao para cruacao dos marcadores
def grupo_mapa(natureza_lesao):
    group = plugins.FeatureGroupSubGroup(mc, natureza_lesao)
    mapa_cluster.add_child(group)
    
    for i in data[data['natureza_lesao'] == natureza_lesao].itertuples():
        folium.Marker(location=[i.latitude, i.longitude], 
                      popup=natureza_lesao, 
                      icon=folium.Icon(color=dic_nl[natureza_lesao], 
                                       icon='bolt',
                                       prefix='fa')).add_to(group)
    

In [10]:
#iterando a funcao sobre o set
for i in dic_nl.keys():
    grupo_mapa(i)

In [11]:
# adicionando menu
folium.LayerControl(collapsed=False).add_to(mapa_cluster);

### Visualizando o mapa

In [12]:
mapa_cluster

### Salvando o mapa

In [120]:
mapa_cluster.save('../examples/mapa_cluster.html')